# Preconditioned Conjugated Gradient for Toeplitz System 

## Form Toelitz Systems

A $n \times n$ matrix $A_n$ is called a Toeplitz matrix if it has the form

$$
A_n = 
\begin{bmatrix}
a_0 & a_{-1} & \cdots & a_{2-n} & a_{1-n} \\
a_1 & a_0 & a_{-1} & \ddots & \vdots \\
\vdots & a_1 & \ddots & \ddots & a_{2-n} \\
a_{n-2} & \ddots & \ddots & a_0 & a_{-1} \\
a_{n-1} & a_{n-2} & \cdots & a_1 & a_0
\end{bmatrix}
$$
In this project, we are solving **symmetric positive definite Toeplitz systems** by preconditioned CG methods using **circulant matrices** as the preconditioners

### Power decay System

$a_k = | k + 1 |^{-p}$ for the lower triangular part of $A_n$, where $p = 2, 1, 1/10, 1/100$

In [5]:
import numpy as np
import time
import scipy
import math

In [6]:
def generate_power_decay_system(n: int, p: np.float64) -> np.ndarray:
    """
    A_n = [
        a_0, a_{-1}, ..., a_{2 - n}, a_{1 - n},
        a_1, a_0, a{-1}, ..., a_{2 - n},
        ...
        a_{n-1}, a_{n-2}, ..., a_1, a_0
    ]
    A:SPD
    a_k = | k + 1 |^(-p) for lower triangular part
    """
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i >= j:
                A[i, j] = abs(i - j + 1) ** (-p)
    # Make A as symmetric
    A += A.T - np.diag(A.diagonal())

    return A

In [17]:
A = generate_power_decay_system(4, 1)
print(A)

[[1.         0.5        0.33333333 0.25      ]
 [0.5        1.         0.5        0.33333333]
 [0.33333333 0.5        1.         0.5       ]
 [0.25       0.33333333 0.5        1.        ]]


### FFT System

$$a_k = \frac{1}{2\pi} \int_{-\pi}^{\pi} f(\theta)e^{-ik\theta}d\theta, k = 0, \pm1, \pm2, ...$$
$$ f(\theta) = \theta^4 + 1, -\pi \leq \theta \leq \pi$$

Since the integral is the formula of continuous fourier transform, we can form the system's first column by using FFT (use $\theta$ as sample parameter), then construct the symmetric Toeplitz matrix: 

In [18]:
def generate_FFT_system(n: int, p) -> np.ndarray:
    """
        A_n = [
        a_0, a_{-1}, ..., a_{2 - n}, a_{1 - n},
        a_1, a_0, a{-1}, ..., a_{2 - n},
        ...
        a_{n-1}, a_{n-2}, ..., a_1, a_0
    ]
    A:SPD
    a_k = 1 / (2pi) * integral_-pi^pi f(theta) e^(-i * k * theta) dtheta
    f(theta) = theta^4 + 1
    """
    f = lambda theta: theta ** 4.0 + 1
    thetas = np.linspace(-np.pi, np.pi, n, endpoint=False)
    first_col = (np.fft.fft(f(thetas))) / np.float64(n)
    A = scipy.linalg.toeplitz(first_col.real, first_col.real)

    return A

In [19]:
A = generate_power_decay_system(4, 1)
print(A)

[[1.         0.5        0.33333333 0.25      ]
 [0.5        1.         0.5        0.33333333]
 [0.33333333 0.5        1.         0.5       ]
 [0.25       0.33333333 0.5        1.        ]]


## CG Algorithm

This is the normal conjugate gradient method with termination criteria $\frac{\Vert r^k \Vert}{\Vert r^0 \Vert} <= 10^{-6}$ :

In [9]:
def conjugate_gradient(A: np.ndarray, b: np.ndarray, max_iterations=10000, tol=1e-6) -> np.ndarray:
    start_time = time.time()
    x = np.zeros_like(b, dtype=np.float64)
    r = b - A @ x
    p = r.copy()

    r0 = r.copy()
    r0_norm = np.linalg.norm(r0)

    for j in range(max_iterations):
        q = A @ p
        alpha = np.dot(r, r) / np.dot(p, q)
        x = x + alpha * p
        r_new = r - alpha * q
        if np.linalg.norm(r_new) / r0_norm < tol:
            print("Number of iterations:", j + 1)
            print("Elapsed time:", time.time() - start_time, "s")
            return x
        beta = np.dot(r_new, r_new) / np.dot(r, r)
        p = r_new + beta * p
        r = r_new
    print("Warning: Solution did not converge after {} iterations".format(max_iterations))
    print("Best approximation obtained within tolerance.")
    print("Elapsed time:", time.time() - start_time, "s")
    return x

## PCG Algorithm

### Circulant Preconditioners

CG can be speed up by using circulant preconditioners. A $n \times n$ matrix $C_n$ is called circulant matrix if  
$$
C_n = 
\begin{bmatrix}
c_0 & c_{-1} & \cdots & c_{2-n} & c_{1-n} \\
c_{1-n} & c_0 & c_{-1} & \cdots & c_{2-n} \\
\vdots & c_{1-n} & c_0 & \cdots & \vdots \\
c_{-2} & \cdots & \cdots & \cdots & c_{-1} \\
c_{-1} & c_{-2} & \cdots & c_{1-n} & c_0 \\
\end{bmatrix}
$$


#### G. Strang's circulant preconditioner

G. Strang’s circulant preconditioner which is defined as $[C_n]_{k,l} = c_{k-l}$ for $0 \leq k, l < n$, where

$$
c_j = \begin{cases} 
a_j & \text{for } 0 \leq j \leq \left\lfloor \frac{n}{2} \right\rfloor \\
a_{j-n} & \text{for } \left\lfloor \frac{n}{2} \right\rfloor < j < n \\
c_{n+j} & \text{for } 0 < -j < n
\end{cases}
$$

where operator $[x]$ returns the closest integer smaller than $x$.

In fact, we don't have to construct complete matrix. Instead, we only need to know matrix's first column, I'll explain why later. 

In [11]:
def get_A_index(k, n) -> tuple:
    """
    This function is used to get the location of the kth element in the A matrix.
    """
    # First row
    if k <= 0:
        row = 0
        col = -k
    # First column
    elif k > 0:
        row = k
        col = 0
    # This is in T_chan's case if k - n = n or -n in this case we use k = 0
    if k == n or k == -n:
        row = col = 0
    return row, col


def G_Strang_first_column(A: np.ndarray) -> np.ndarray :
    n = A.shape[0]
    D = np.zeros(n, dtype=np.float64)
    for k in range(n):
        if k <= math.floor(n / 2):
            D[k] = A[get_A_index(k, n)]
        else:
            D[k] = A[get_A_index(k - n, n)]
    return D

In [21]:
M = G_Strang_first_column(A)
print(M)

[1.         0.5        0.33333333 0.5       ]


#### T. Chan's circulant preconditioner

T. Chan’s circulant preconditioner

$$
c_j = \begin{cases} 
\frac{(n-j)a_j + ja_{j-n}}{n} & \text{for } 0 \leq j < n \\
c_{n+j} & \text{for } 0 < -j < n 
\end{cases}
$$


Similarly, we only need to form out the first column of the matrix

In [12]:
def T_Chan_first_column(A: np.ndarray) -> np.ndarray:
    n = A.shape[0]
    D = np.zeros(n, dtype=np.float64)
    for k in range(n):
        aj = A[get_A_index(k, n)]
        aj_minus_n = A[get_A_index(k - n, n)]
        D[k] = ((n - k) * aj + k * aj_minus_n) / n
    return D

In [20]:
M = T_Chan_first_column(A)
print(M)

[1.         0.4375     0.33333333 0.4375    ]


### Precondition strategy

In my method, I use **M-inner product** strategy to do the preconditioning, which means in every step I have to solve a linear system $Mz = r$, which is the critical part of the PCG algorithm that determines the time performance. To solve this linear system efficiently, we must use the property of the circulant matrices:

In fact, circulant matrices are diagonalized by the Fourier matrix $F_n$, i.e. 
$$
C_n = F_n^{*} \Lambda_n F_n
$$

where $ [F]_{j,k} = \frac{1}{\sqrt{n}} e^{-2\pi ijk/n}$ for $0 \leq j,k \leq n - 1$ and $\Lambda_n$ is a diagonal matrix holding the eigenvalues of $C_n$. $\Lambda_n$ can be obtained in $O(nlogn)$ operations by taking FFT of the first column. An intuition for this is that the Fourier Transform is designed to be working with periodic functions and signals, and circuland matrices with their periodic row shifts, fit neatly into this framework. The columns of DFT matrix(computed via FFT) align perfectly as the eigenvectors of any circulant matrix, making FFT an ideal tool for diagonalization.

Once $\Lambda_n$ is obtained, the product $C_n^{-1}\vec{y}$ can be computed by FFT using $O(nlogn)$ operations for any given vector $\vec{y}$, this makes solve $Mz=r$ very quickly. 

Here is the proof of why we can use FFT to compute $C_n^{-1}\vec{y}$:

$$
C_n^{-1}\vec{y} \\
= (F_n^{*} \Lambda_n F_n)^{-1} \vec{y} \\
= F_n^{*} \ \Lambda_n^{-1} F_n \vec{y} \\
= F_n^{*} \ \Lambda_n^{-1} FFT(\vec{y}) \\
= F_n^{*} \ FFT(\text{first column of } C_n)^{-1} FFT(\vec{y}) \\
= IFFT(FFT(\text{first column of } C_n)^{-1} \ FFT(\vec{y})) \\
$$

Thus, to compute $C_n^{-1} \vec{y}$, there are three steps:
1. Compute the DFT of $\vec{y}$ (using FFT)
2. Find the eigenvalues of $C_n$ by computing the DFT of its first column, denoted as $\Lambda_n$
3. Multiply inverse of $\Lambda_n$ with the DFT of \vec{y}
4. Apply the IDFT of the result

As a result, we can implment PCG easily:

In [13]:
def preconditioned_conjugate_gradient(A: np.ndarray, b: np.ndarray, preconditioner_fun, max_iterations=10000, tol=1e-6) -> np.ndarray:
    """
    Solve Ax = b using the preconditioned conjugate gradient method.
    Use CG in M-inner product
    :param M: preconditioner
    """
    start_time = time.time()

    C_first_column = preconditioner_fun(A)
    D_inv = 1 / np.fft.fft(C_first_column)

    x = np.zeros_like(b, dtype=np.float64)
    r = b - A @ x
    # Solve Mz = r
    z = np.fft.ifft(D_inv * np.fft.fft(r)).real
    p = z.copy()

    r0 = r.copy()
    r0_norm = np.linalg.norm(r0)

    for j in range(max_iterations):
        q = A @ p
        alpha = np.dot(r, z) / np.dot(p, q)
        x = x + alpha * p
        r_new = r - alpha * q
        if np.linalg.norm(r_new) / r0_norm < tol:
            print("Number of iterations:", j + 1)
            print("Elapsed time:", time.time() - start_time, "s")
            return x
        # Solve Mz = r
        z_new = np.fft.ifft(D_inv * np.fft.fft(r_new)).real
        beta = np.dot(r_new, z_new) / np.dot(r, z)
        p = z_new + beta * p
        r = r_new
        z = z_new
    print("Warning: Solution did not converge after {} iterations".format(max_iterations))
    print("Best approximation obtained within tolerance.")
    print("Elapsed time:", time.time() - start_time, "s")
    return x


## Remarks